# フローベースモデル

潜在変数から観測変数は，決定論的な写像 & 可逆であるとする．

潜在変数と観測変数は，全単射の関係である．てことは潜在変数の次元と観測変数の次元は等しい．

わかっているのは，潜在変数の確率分布と，変換器$f$

確率密度関数の変数変換の公式によって，潜在変数の確率密度関数から，観測変数の確率密度関数を求めることができる．

$$
p(x)dx = p(y)dy
$$
確率密度関数の積分は，変数変換前後で等しい．

確率密度関数なので，変数を変換した後でも，総面積は変わらない．それが微少量に着目していたとしてもね．

んでこの式を変形すると，$$p(y) = p(x) \left| \frac{dx}{dy} \right|$$となる．

では，これの対数尤度を考える．

$$
\log p(y) = \log p(x) + \log \left| \frac{dx}{dy} \right|
$$

これだけ，多変量だとして，ヤコビアンを導入すると，

$$
\log p(\boldsymbol{x}) = \log p(f^{-1}(\boldsymbol{x})) + \log \left| \det \frac{df^{-1}(\boldsymbol{x})}{d\boldsymbol{x}} \right|
$$

フローベースモデルのジレンマ

- $f$をリッチにしたい．
    - しかし，fは可逆である必要がある．
    - ヤコビ行列式が計算できなければならない．

これをどう解決するかがフローベースモデルのテーマとなる．

- 正規化フロー
f自体は，単純な変換しかできないので，単純な変換を繰り返し適用(関数の合成)をすることで複雑な分布を表現する．
$$
\boldsymbol{x} = \boldsymbol{z}_K = f_K \circ f_{K-1} \circ \cdots \circ f_1(\boldsymbol{z}_0)
$$
これを，パスと呼ぶ．

可逆な$f$による$z_0$から合成変換のパスのことをフローと言い，フローによって有効な分布が構成される場合は正規化フローと言う．

$\boldsymbol{x}$をサンプリングするためには，$\boldsymbol{z}_0$をサンプリングする．

で，最初0の分布が簡単なので，サンプリングは容易．フローに通しちゃえばいいだけだから．

それは以下．

$$
\begin{align*}
\log p(\boldsymbol{x}) = \log p(\boldsymbol{z}_0) &= \log p_{K-1}(\boldsymbol{z}_{K-1}) - \log \left| \det \frac{df_{K}}{d \boldsymbol{z}_{K-1}} \right| \\
&= \log p_{K-2}(\boldsymbol{z}_{K-2}) - \log \left| \det \frac{df_{K-1}}{d \boldsymbol{z}_{K-2}} \right| - \log \left| \det \frac{df_{K}}{d \boldsymbol{z}_{K-1}} \right| \\
&= \cdots \\
&= \log p_0(\boldsymbol{z}_0) - \sum_{i=1}^{K} \log \left| \det \frac{df_i}{d \boldsymbol{z}_{i-1}} \right|
\end{align*}
$$

各層で$\log$ヤコビアンを計算する．

フローを用いることのメリットは，

推論が，逆変換を用いてできること．変換を遡れる．

$x$が与えられたときに，$z$を求めることができる．

今まではVAEだったらエンコーダを作ったりとかで工夫した．GANだったらdiscriminatorを作ったりとかで工夫した．

要は二つモデルを作らなければいけなかったんよね．

だけどフローベースであれば，可逆なのでモデルは一個で良い．

推論を$f$，瀬性を$f^{-1}$で表現できる．

そのようなモデル$f$を設計するためには，
- 可逆であること．(全単射)
- ヤコビアンが簡単に計算できること．

このような$f$をニューラルネットワークでどう表現するかが課題である．

$f$はどうモデリングすれば良いのだろうか．

まず，$f$は単純で良いのだが，$x$の次元×$x$の次元のヤコビアンを計算することは変わらない．

これをどう楽していこうかね．

- 三角行列

ヤコビ行列が三角行列であれば，行列式は対角成分の積で求められる．

よって，ヤコビ行列が三角行列になるような可逆変換を考える．

- カップリング層

入力$\boldsymbol{x}\in \mathbb{R}^D$を，二つのブロックに分ける．
- $\boldsymbol{x} = [\boldsymbol{x}_{1:d}:\boldsymbol{x}_{d+1:D}] = [\boldsymbol{x}_{I_1}:\boldsymbol{x}_{I_2}]$

それぞれのブロックに対して，次のような変換を考える．(カップリング層)

$$

\begin{align*}
\boldsymbol{y}_{I_1} &= \boldsymbol{x}_{I_1} \\
\boldsymbol{y}_{I_2} &= g(\boldsymbol{x}_{I_2} ; m(\boldsymbol{x}_{I_1}))
\end{align*}

$$

- $g:\mathbb{R}^{D-d} \times \mathbb{R}^d \to \mathbb{R}^{D-d}$は可逆関数
- $m$は任意の関数(カップリング関数と言う)

この変換をすると，三角行列になる，ヤコビ行列式が計算しやすくなる．

ポイント：逆変換において，$g$が可逆である必要はあるが，$m$は可逆である必要はない，だからDNNとかを使えちゃう．

- NICE(Non-linear Independent Component Estimation) 2014年あたり．

加法カップリングを利用したモデル．($g(\boldsymbol{a};\boldsymbol{b})=\boldsymbol{a}+\boldsymbol{b}$)

-この中は省略-

結果的にヤコビ行列式は1になる．

変換前後で体積が不変であるということ．

学習がめっちゃシンプルになるのは良いが，フローの過程で分布が変化していないのと同義になる．

- NICEの工夫
    - スケーリング行列を導入(対角行列$S$を導入，行列式が対角成分の積になる．)

$$
\log p(\boldsymbol{x}) = \sum_{i=1}^{D}[\log p_i(f_i(\boldsymbol{x}) + \log \left| S_{ii} \right|]
$$

- 潜在変数の事前分布
    - VAEなどと同様，ガウス分布や，ロジスティック分布などを使用．

- RealNVP(Real-valued Non-Volume Preserving) 2017年あたり．

アフィンカップリングを利用
- $g(\boldsymbol{a};\boldsymbol{b})=\boldsymbol{a} \odot \boldsymbol{b}_s + \boldsymbol{b}_t, \boldsymbol{b}_s = \exp(s(\boldsymbol{x}_{I1})), \boldsymbol{b}_t = t(\boldsymbol{x}_{I1})$
- ただし，$s\mathbb{R}^{d} \to \mathbb{R}^{D-d}, t:\mathbb{R}^{d} \to \mathbb{R}^{D-d}$
    - $s$や$t$にはDNNなどを用いる．

アフィンカップリングによるカップリング層は，...略

NICEと異なり，行列式は1ではない．変換前後で体積が変わっている(分布がちゃんと変化している)

基本的な考え方．

- ヤコビ行列を対角行列にしたい．
    - そのためにカップリング層を導入する．
       - その中で$g$や$m$を考える．
            - $g$は可逆にしたい(シンプルにしたい．)
                - シンプルしすぎると分布が変わらないので，いい感じに工夫する．(具体的には，アフィン変換とかね．)
            - $m$はなんでもいい．

問題があって$x_1, x_2$に2ブロックに分割したとき，片方は変換がかかって，もう片方はかからないよね．

これは，半分が変換されて，半分が変換されないって感じでよくない．

これを解決するために，入れ替える．

交互に変換するって感じ．

片方を固定して，片方を変換．それを繰り返す．

これはマスキングとも捉えられる．

- Multi-Scale Architecture

交互のカップリング$K$回を行う．

- Glow 2018年あたり．

Multi-Scale Architectureと同じなんだけど，複雑に交互にマスキングとかはしない，シンプルにした．

- フローベースモデルのまとめ
- 利点
    - 厳密な推論，対数尤度の評価ができる．
        - 既存の深層生成モデルでは困難
        - 画像生成以外いにも，異常検知，幅広い観点での利用が期待．
    - 推論，生成が効率的，(可逆)
    - 潜在変数内での操作ができる．
- 欠点
    - アーキテクチャに制限がある．
        - 逆変換やヤコビ行列の計算が容易である層を利用する必要がある．
        - 層を増やしても次元は変わらない．
    - 各フローでヤコビ行列などをとっておく必要があるので，全体としてメモリを食う．

# 自己回帰モデル

$\boldsymbol{x}=[x_1, \ldots, x_D]$について，尤度を条件付き分布の積で構成．

$$
p(\boldsymbol{x}) = \prod_{d=1}^{D} p(x_d | x_{<d})
$$

フローベース同様，厳密に尤度を計算できる．

これをニューラルネットで構築したいわね．

でも，サンプリングが超大変．$x_1$から順番にサンプリングしないといけない．次元が大きくなると，画像の左上からサンプリングするみたいになって大変だね．

自己回帰モデルでは，どのようにっ条件付き分布を効率的に計算できるかがポイント．

1ピクセルごとに計算するのはきつい．

- MADE(Masked Autoencoder for Distribution Estimation)

オートエンコーダの各層にマスクをかけて，先の要素以外から条件づけられるように構成する．

- 隠れ層に適当な番号をつける．
- 自分自身の番号より大きいユニットからの経路を遮断するようにマスクを1にする．

→一回の順伝播で，自己回帰の全条件付き分布の出呂0区を得ることができる．(めっちゃすごい．これまではこれがきつかった)

- PixelCNN/PixelRNN

画像を上から下，左から右に生成する．
- 自分より前のピクセルが文脈，条件となる．
- 文脈だけを見る方法は3種類提案．
    - PixelCNN
        - マスク付きの畳み込み．
            - 高速に計算できるものの，文脈を全部見ることができないため，尤度の推定精度が低い．
    - PixelRNN
        - Row LSTM
            - 上から下に1D Conv+LSTMを使う．
            - 上方向の文脈は全て見ることができるが，速度が遅い．
        - Diagonal BiLSTM
            - 正方形に文脈を見るために，各行を1ピクセルずつずらした後に1D Conv+LSTM(Sekwing)
            - 文脈を全てみれるが，Row LSTMよりも速度が遅い．

同じ文脈を保って多層構造にする．

これらは，高次元画像を生成できるが，一貫性を捉えることができていない．

- WaveNet
    - 1-Dの音声信号に対する，pixelCNNに基づく自己回帰モデル
        - 綺麗な音声合成ができるということで話題．
    - Casual Convolution
        - pixelCNNと同様に，未来の情報を使わないようにする．
        - ただし，マスクではなく，単純にタイムステップの分をずらす．
    - Dilated Convolution
        - convだと限られた受容野からしか情報を受け取れない．
        - そこで，広い範囲から飛び飛びに入力をとって畳みこむ．
            - 層が大きいほど飛ぶ大きさを大きくする．

自己回帰モデルでは，いかに遠いところからの情報を持ってくるかで工夫をしていた．

# 自己回帰フロー